In [28]:
import os
# Initialise our neural network model as a sequential network
from keras.models import Sequential
# Convolution operation
from keras.layers import Conv2D 
 # Maxpooling function
from keras.layers import MaxPooling2D
# Converting 2D arrays into a 1D linear vector.
from keras.layers import Flatten 
 # Perform the full connection of the neural network
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import display
from PIL import Image
import cv2
import numpy as np
from sklearn.metrics import accuracy_score
from skimage import io, transform

In [17]:
#CNN model
def cnn_classifier():
    model = Sequential()
    
    model.add(Conv2D(32, (3,3), input_shape = (50, 50, 3), padding='same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    
    model.add(Conv2D(64, (3,3), padding='same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    
    model.add(Flatten())
    model.add(Dense(500, activation = 'relu'))
    model.add(Dense(2, activation = 'softmax'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    print(model.summary())
    return model

In [18]:
#Reshaping all images into same size
def reshaped_image(image):
    return transform.resize(image,(50, 50, 3)) # (cols (width), rows (height))


In [19]:
#function to load the training data
def load_images_from_folder():
    Images = os.listdir("./dataset/train1/")
    train_images = []
    train_labels = []
    for image in Images:
            l = [0,0] # [cat,dog]
            if image.find('cat') != -1:
                path = os.path.join("./dataset/train1/", image)
                img = cv2.imread(path)
                train_images.append(reshaped_image(img))
                l = [1,0] 
                train_labels.append(l)
            if image.find('dog') != -1:
                path = os.path.join("./dataset/train1/", image)
                img = cv2.imread(path)
                train_images.append(reshaped_image(img))
                l = [0,1] 
                train_labels.append(l)
    return np.array(train_images), np.array(train_labels)

In [20]:
def train_test_split(train_data, train_labels, fraction):
    index = int(len(train_data)*fraction)
    return train_data[:index], train_labels[:index], train_data[index:], train_labels[index:]


In [21]:
train_data, train_labels = load_images_from_folder()
fraction = 0.73

/home/vs/anaconda3/envs/ml/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/vs/anaconda3/envs/ml/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [22]:
train_data, train_labels, test_data, test_labels = train_test_split(train_data, train_labels, fraction)
print ("Train data size: ", len(train_data))
print ("Test data size: ", len(test_data))

Train data size:  2920
Test data size:  1080


In [23]:
model = cnn_classifier()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 50, 50, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               5408500   
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 1002      
Total para

In [24]:
print ("Train data shape: ", train_data.shape)
print ("Test data shape: ", test_data.shape)

Train data shape:  (2920, 50, 50, 3)
Test data shape:  (1080, 50, 50, 3)


In [25]:
idx = np.random.permutation(train_data.shape[0])
model.fit(train_data[idx], train_labels[idx], batch_size = 64, epochs = 10)
predicted_test_labels = np.argmax(model.predict(test_data), axis=1)
test_labels = np.argmax(test_labels, axis=1)

Epoch 1/10
2920/2920 [==============================] - 46s - loss: 0.7814 - acc: 0.5216    
Epoch 2/10
2920/2920 [==============================] - 46s - loss: 0.6532 - acc: 0.6158    
Epoch 3/10
2920/2920 [==============================] - 53s - loss: 0.5849 - acc: 0.6932    
Epoch 4/10
2920/2920 [==============================] - 54s - loss: 0.5233 - acc: 0.7442    
Epoch 5/10
2920/2920 [==============================] - 58s - loss: 0.4638 - acc: 0.7818    
Epoch 6/10
2920/2920 [==============================] - 61s - loss: 0.3925 - acc: 0.8130    
Epoch 7/10
2920/2920 [==============================] - 51s - loss: 0.3305 - acc: 0.8521    
Epoch 8/10
2920/2920 [==============================] - 42s - loss: 0.2602 - acc: 0.8860    
Epoch 9/10
2920/2920 [==============================] - 42s - loss: 0.2109 - acc: 0.9192    
Epoch 10/10
2920/2920 [==============================] - 43s - loss: 0.1634 - acc: 0.9411    


In [26]:
print ("Actual test labels:\n", test_labels)
print ("Predicted test labels:\n", predicted_test_labels)

Actual test labels:
 [0 0 0 ... 1 1 0]
Predicted test labels:
 [0 0 1 ... 1 1 0]


In [27]:
print ("Accuracy score:", accuracy_score(test_labels, predicted_test_labels))

Accuracy score: 0.7453703703703703
